In [1]:
%load_ext pycodestyle_magic
%flake8_on
# la convention PEP8 est ainsi testée

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from itertools import product, combinations
from sklearn.preprocessing import StandardScaler, QuantileTransformer, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score as sisc
from sklearn.metrics import davies_bouldin_score as dbsc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# home made
import functions
from functions import distribution
import time
%matplotlib inline

7:80: E501 line too long (98 > 79 characters)
8:80: E501 line too long (92 > 79 characters)
1: E999 SyntaxError: invalid syntax


In [7]:
# read the data
data = pd.read_csv('Data/data.csv')

In [9]:
# show the list of features
data.columns

Index(['order_purchase_year', 'order_purchase_month', 'purchase_time_zone_cat',
       'effective_delivery_time', 'delivery_vs_estimated',
       'delivery_vs_estimated_lvl', 'items_qty', 'products_qty', 'total_price',
       'freight_percentage', 'freight_percentage_lvl', 'total_price_lvl',
       'charmed_price_cat', 'product_name_length',
       'product_description_length', 'product_photos_qty', 'product_weight_g',
       'product_size', 'product_photos_qty_lvl', 'product_cat',
       'main_payment_type', 'payment_sequence_size',
       'payment_installments_size', 'payment_sequence_size_cat',
       'payment_installment_size_cat', 'main_payment_type_cat', 'payment_lvl',
       'seller_state', 'customer_unique_id', 'customer_state',
       'cust_sell_dist', 'cust_sell_dist_lvl', 'review_answer_delay',
       'review_answer_delay_lvl', 'product_review_mean',
       'product_review_mean_lvl', 'customer_review_mean', 'review_gap',
       'review_gap_lvl'],
      dtype='object')

In [10]:
# subset of numerical_features
numerical_features = data.select_dtypes(include='number').columns

In [11]:
# show subset
numerical_features

Index(['order_purchase_year', 'order_purchase_month',
       'effective_delivery_time', 'delivery_vs_estimated', 'total_price',
       'freight_percentage', 'product_name_length',
       'product_description_length', 'product_photos_qty', 'product_weight_g',
       'product_size', 'product_photos_qty_lvl', 'payment_sequence_size',
       'payment_installments_size', 'cust_sell_dist', 'review_answer_delay',
       'product_review_mean', 'customer_review_mean', 'review_gap'],
      dtype='object')

In [13]:
# create a dedicated df
data_filtered = data.filter(numerical_features)
data_filtered.shape

(87086, 19)

In [15]:
initial_features = data_filtered.columns.tolist()
initial_features

['order_purchase_year',
 'order_purchase_month',
 'effective_delivery_time',
 'delivery_vs_estimated',
 'total_price',
 'freight_percentage',
 'product_name_length',
 'product_description_length',
 'product_photos_qty',
 'product_weight_g',
 'product_size',
 'product_photos_qty_lvl',
 'payment_sequence_size',
 'payment_installments_size',
 'cust_sell_dist',
 'review_answer_delay',
 'product_review_mean',
 'customer_review_mean',
 'review_gap']

In [16]:
# define transformer for initial_features
numerical_pipeline = Pipeline(
    steps=[
        ('rankgauss', QuantileTransformer(n_quantiles=100,
                                          random_state=111,
                                          output_distribution='normal')),
        ('scaler', StandardScaler())
    ]
)

In [17]:
mytransformer = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features)])

In [18]:
mytransformer.fit(data_filtered)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('rankgauss',
                                                  QuantileTransformer(n_quantiles=100,
                                                                      output_distribution='normal',
                                                                      random_state=111)),
                                                 ('scaler', StandardScaler())]),
                                 Index(['order_purchase_year', 'order_purchase_month',
       'effective_delivery_time', 'delivery_vs_estimated', 'total_price',
       'freight_percentage', 'product_name_length',
       'product_description_length', 'product_photos_qty', 'product_weight_g',
       'product_size', 'product_photos_qty_lvl', 'payment_sequence_size',
       'payment_installments_size', 'cust_sell_dist', 'review_answer_delay',
       'product_review_mean', 'customer_review_mean', 'review_gap'],
      dtype='object'))])

In [54]:
data_preped = pd.DataFrame(mytransformer.transform(data_filtered))

In [56]:
data_preped.columns = initial_features

In [58]:
data_preped.describe()

,order_purchase_year,order_purchase_month,effective_delivery_time,delivery_vs_estimated,total_price,freight_percentage,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_size,product_photos_qty_lvl,payment_sequence_size,payment_installments_size,cust_sell_dist,review_answer_delay,product_review_mean,customer_review_mean,review_gap
count,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04,8.708600e+04
mean,6.062235e-14,3.683066e-14,-1.187196e-15,9.284325e-15,1.489213e-16,1.814863e-16,-1.791533e-16,6.712368e-16,-7.037233e-15,-2.141059e-15,-1.103483e-15,2.511412e-15,1.228637e-14,-1.964595e-14,-2.047142e-16,1.174484e-14,9.757014e-15,-1.059085e-14,3.303604e-15
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,-2.623186e+00,-2.523987e+00,-5.193537e+00,-5.274264e+00,-5.289825e+00,-5.060541e+00,-5.314113e+00,-5.232591e+00,-1.011288e+00,-5.308737e+00,-5.279569e+00,-9.632880e-01,-1.759306e-01,-1.007506e+00,-5.251989e+00,-1.738263e+00,-2.453369e+00,-2.135382e+00,-5.318465e+00
25%,-1.117117e+00,-3.534248e-01,-7.009177e-01,-7.404246e-01,-6.775506e-01,-6.454525e-01,-7.249562e-01,-6.840505e-01,-1.011288e+00,-6.367107e-01,-6.994170e-01,-9.632880e-01,-1.759306e-01,-1.007506e+00,-6.820158e-01,2.713963e-01,-5.699537e-01,-8.284289e-01,-6.926613e-01
50%,8.949978e-01,5.323856e-02,4.690491e-02,-3.699938e-02,7.194080e-03,1.294594e-02,3.240355e-02,1.118164e-03,7.881394e-01,2.150402e-02,-4.502342e-03,6.392514e-01,-1.759306e-01,7.649595e-01,-2.000162e-04,2.713963e-01,-2.876271e-01,7.753567e-01,-2.386314e-01
75%,8.949978e-01,3.315150e-01,6.775521e-01,7.103477e-01,6.821807e-01,6.636270e-01,7.048658e-01,6.733699e-01,9.328699e-01,6.841145e-01,6.861673e-01,7.681460e-01,-1.759306e-01,9.504758e-01,6.833031e-01,7.003100e-01,-6.086674e-03,7.753567e-01,6.810004e-01
max,8.949978e-01,2.605482e+00,5.186210e+00,5.277359e+00,5.291235e+00,5.081089e+00,5.327094e+00,5.229732e+00,2.435184e+00,5.300078e+00,5.278243e+00,2.106081e+00,8.053318e+00,2.444662e+00,5.253704e+00,2.489171e+00,1.879858e+00,7.753567e-01,5.311182e+00


In [59]:
# echantillonnage (10%)
data_sample = pd.DataFrame(data_preped).sample(
    int(data.shape[0]/10),
    random_state=42)
data_sample.shape

(8708, 19)

In [120]:
# reduction à 3 features
test_features = ['product_review_mean',
                 'customer_review_mean',
                 'review_gap']

In [121]:
# define clusterer
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters)

In [75]:
data_sample[test_features]

,total_price,delivery_vs_estimated,review_gap
57393,-1.860638,-0.435515,-1.001312
3137,-1.443629,-0.298213,-1.343230
58640,-1.908564,-0.036999,-1.434387
1368,1.509393,1.325664,0.791097
74708,-0.923687,0.274571,1.829320
...,...,...,...
86690,0.361535,0.710348,0.686991
2868,-0.002492,-1.210416,0.184120
10474,-1.192969,-0.901030,1.177009
74766,0.615894,0.274571,1.914394


In [76]:
# fit to the data
kmeans.fit_predict(data_sample[test_features])

array([0, 0, 0, ..., 2, 1, 0])

In [94]:
# get the silhouette score
sisc(data_sample[test_features], kmeans.fit_predict(data_sample[test_features]))

0.23284920400143563

In [122]:
# create the forward selector

def sisc_forward_selection(data, score_threshold):
    """
    give the "best features",
    according to silhouette_score
    over a given score_threshold
    """

    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features) > 0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_sisc = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            new_sisc[new_column] = sisc(
                data[best_features+[new_column]],
                kmeans.fit_predict(data[best_features+[new_column]])
            )
        max_sisc = new_sisc.max()
        if(max_sisc > score_threshold):
            best_features.append(new_sisc.idxmax())
        else:
            break
    return best_features

In [124]:
sisc_forward_selection(data_sample[test_features], 0.3)

['customer_review_mean', 'product_review_mean', 'review_gap']

In [123]:

def dbsc_forward_selection(data, score_threshold):
    """
    give the "best features",
    according to davies-bouldin index
    over a given score_threshold
    """

    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features) > 0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_dbsc = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            new_dbsc[new_column] = dbsc(
                data[best_features+[new_column]],
                kmeans.fit_predict(data[best_features+[new_column]])
            )
        min_dbsc = new_dbsc.min()
        if(min_dbsc < score_threshold):
            best_features.append(new_dbsc.idxmin())
        else:
            break
    return best_features

In [125]:
dbsc_forward_selection(data_preped[test_features], 1)

['customer_review_mean', 'product_review_mean', 'review_gap']

In [127]:
product_features = ['product_name_length',
                    'product_description_length',
                    'product_photos_qty',
                    'product_weight_g',
                    'product_size']

In [130]:
dbsc_forward_selection(data_sample[product_features], 1)

['product_photos_qty', 'product_description_length']

In [131]:
sisc_forward_selection(data_sample[product_features], 0.3)

['product_photos_qty', 'product_size', 'product_weight_g']